---
draft: true
title: mesa tests
---


So this is supposed to become a python Shiney app for the schelling model
using mesa modeling environment.



But I soon decided to just try to get the demo notebook from [here](https://github.com/projectmesa/mesa-examples/blob/main/examples/schelling/analysis.ipynb) running!

```python
#| label: installation
#| eval: False

# This will install or update the "mese" package
!pip install --quiet --upgrade mesa 
# This will install the "mesa_models" package
!pip install -U -e git+https://github.com/projectmesa/mesa-examples#egg=mesa-models
```


In [ ]:
import mesa
#| label: importing packages
#| eval: True
#| warning: false

# Data visualization tools.
import seaborn as sns

# Has multi-dimensional arrays and matrices. Has a large collection of
# mathematical functions to operate on these arrays.
import numpy as np

# Data manipulation and analysis.
import pandas as pd

## Create The Agent


In [ ]:
#| label: create agent
class MoneyAgent(mesa.Agent):
    """An agent with fixed initial wealth."""

    def __init__(self, unique_id, model):
        # Pass the parameters to the parent class.
        super().__init__(unique_id, model)

        # Create the agent's variable and set the initial values.
        self.wealth = 1


    def move(self):
        possible_steps = self.model.grid.get_neighborhood(
            self.pos,
            moore=True,
            include_center=False)
        new_position = self.random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)

    def give_money(self):
        cellmates = self.model.grid.get_cell_list_contents([self.pos])
        if len(cellmates) > 1:
            other = self.random.choice(cellmates)
            other.wealth += 1
            self.wealth -= 1


    def step(self):
        # The agent's step will go here.
        if self.wealth > 0:
            self.give_money()


## Create Model


In [ ]:
#| label: create model

def compute_gini(model):
    agent_wealths = [agent.wealth for agent in model.schedule.agents]
    x = sorted(agent_wealths)
    N = model.num_agents
    B = sum(xi * (N - i) for i, xi in enumerate(x)) / (N * sum(x))
    return 1 + (1 / N) - 2 * B


class MoneyModel(mesa.Model):
    """A model with some number of agents."""

    def __init__(self, N, width, height):
        super().__init__()
        self.num_agents = N
        # create a grid
        self.grid = mesa.space.MultiGrid(width, height, True)
        # Create scheduler and assign it to the model
        self.schedule = mesa.time.RandomActivation(self)

        # Create agents
        for i in range(self.num_agents):
            a = MoneyAgent(i, self)            
            # Add the agent to the scheduler
            self.schedule.add(a)
            # Add the agent to a random grid cell
            x = self.random.randrange(self.grid.width)
            y = self.random.randrange(self.grid.height)
            self.grid.place_agent(a, (x, y))
        
        self.datacollector = mesa.DataCollector(
            model_reporters={"Gini": compute_gini}, agent_reporters={"Wealth": "wealth"}
        )

    def step(self):
        """Advance the model by one step."""

        # The model's step will go here for now this will call the step method of each agent and print the agent's unique_id
        self.datacollector.collect(self)
        self.schedule.step()


## Running the Model


In [ ]:
#from money_model import MoneyModel

model = MoneyModel(100, 10, 10)
for i in range(20):
    model.step()


In [ ]:
agent_wealth = [a.wealth for a in model.schedule.agents]
# Create a histogram with seaborn
g = sns.histplot(agent_wealth, discrete=True)
g.set(
    title="Wealth distribution", xlabel="Wealth", ylabel="Number of agents"
);  # The semicolon is just to avoid printing the object representation

In [ ]:
all_wealth = []
# This runs the model 100 times, each model executing 10 steps.
for j in range(100):
    # Run the model
    model = MoneyModel(100, 10, 10)
    for i in range(10):
        model.step()

    # Store the results
    for agent in model.schedule.agents:
        all_wealth.append(agent.wealth)

# Use seaborn
g = sns.histplot(all_wealth, discrete=True)
g.set(title="Wealth distribution", xlabel="Wealth", ylabel="Number of agents");

In [ ]:
agent_counts = np.zeros((model.grid.width, model.grid.height))
for cell_content, (x, y) in model.grid.coord_iter():
    agent_count = len(cell_content)
    agent_counts[x][y] = agent_count
# Plot using seaborn, with a size of 5x5
g = sns.heatmap(agent_counts, cmap="viridis", annot=True, cbar=False, square=True)
g.figure.set_size_inches(4, 4)
g.set(title="Number of agents on each cell of the grid");

In [ ]:
model = MoneyModel(100, 10, 10)
for i in range(100):
    model.step()

In [ ]:
gini = model.datacollector.get_model_vars_dataframe()
# Plot the Gini coefficient over time
g = sns.lineplot(data=gini)
g.set(title="Gini Coefficient over Time", ylabel="Gini Coefficient");

In [ ]:
agent_wealth = model.datacollector.get_agent_vars_dataframe()
agent_wealth.head()

In [ ]:
last_step = agent_wealth.index.get_level_values("Step").max()
end_wealth = agent_wealth.xs(last_step, level="Step")["Wealth"]
# Create a histogram of wealth at the last step
g = sns.histplot(end_wealth, discrete=True)
g.set(
    title="Distribution of wealth at the end of simulation",
    xlabel="Wealth",
    ylabel="Number of agents",
);

In [ ]:
agent_list = [3, 14, 25]

# Get the wealth of multiple agents over time
multiple_agents_wealth = agent_wealth[
    agent_wealth.index.get_level_values("AgentID").isin(agent_list)
]
# Plot the wealth of multiple agents over time
g = sns.lineplot(data=multiple_agents_wealth, x="Step", y="Wealth", hue="AgentID")
g.set(title="Wealth of agents 3, 14 and 25 over time");

We can also plot the average of all agents, with a 95% confidence interval for that average.


In [ ]:
# Transform the data to a long format
agent_wealth_long = agent_wealth.T.unstack().reset_index()
agent_wealth_long.columns = ["Step", "AgentID", "Variable", "Value"]
agent_wealth_long.head(3)

# Plot the average wealth over time
g = sns.lineplot(data=agent_wealth_long, x="Step", y="Value", errorbar=("ci", 95))
g.set(title="Average wealth over time")

Which is exactly 1, as expected in this model, since each agent starts with one wealth unit, and each agent gives one wealth unit to another agent at each step.


You can also use pandas to export the data to a CSV (comma separated value), which can be opened by any common spreadsheet application or opened by pandas.

If you do not specify a file path, the file will be saved in the local directory. After you run the code below you will see two files appear (model_data.csv and agent_data.csv)


In [ ]:
#| label: batch_run

# save the model data (stored in the pandas gini object) to CSV
gini.to_csv("model_data.csv")

# save the agent data (stored in the pandas agent_wealth object) to CSV
agent_wealth.to_csv("agent_data.csv")

## designing a Shineylive app

the MVP app should have:

1. [] a UI with controls for the x,y, n - number of steps, 
2. a simulate button
3. a panel with with the seaborn chart for map
4. 
3. a panel with the graph of the wealth distribution results tab with
3. a panel with the wealth over time of top middle and poorest agents 


```{shinylive-python}
#| standalone: true
#| 
from shiny import App, render, ui
import mesa

app_ui = ui.page_fluid(
    ui.input_slider("n", "n", 0, 100, 100),
    ui.input_slider("x", "y", 0, 10, 10),
    ui.input_slider("y", "y", 0, 10, 10),
    ui.output_text_verbatim("txt"),
)

def server(input, output, session):
    @output
    @render.text    
    def txt():
        model = MoneyModel(n, x, y)
        for i in range(100):
            model.step()

        last_step = agent_wealth.index.get_level_values("Step").max()
        end_wealth = agent_wealth.xs(last_step, level="Step")["Wealth"]
        return f"the final wealth is {end_wealth}"
        # Create a histogram of wealth at the last step
        #g = sns.histplot(end_wealth, discrete=True)
        #g.set(
        #    title="Distribution of wealth at the end of simulation",
        #    xlabel="Wealth",
        #    ylabel="Numeylive ber of agents",
        #);

app = App(app_ui, server)

```